In [11]:
from symred.symplectic import ObservableGraph

In [12]:
from symred.chem import MoleculeBuilder
from symred.projection import QubitTapering, CS_VQE
from symred.symplectic import QuantumState
import json

In [13]:
with open('data/molecule_data.json', 'r') as jfile:
    molecule_geometries = json.load(jfile)

In [14]:
# Set molecule parameters
speciesname = 'Be_STO-3G_SINGLET'
#print(speciesname in scoring_data)
mol_data = molecule_geometries[speciesname]
if 'name' in mol_data:
    print(mol_data['name'])
    
atoms  = mol_data['atoms']
coords = mol_data['coords']
basis  = mol_data['basis'] #'6-31g' #'cc-pVDZ' #
spin   = mol_data['multiplicity']
charge = mol_data['charge']
geometry = list(zip(atoms, coords))
molecule = MoleculeBuilder(geometry=geometry, charge=charge, basis=basis)

Molecule geometry:
Be	0	0	0

HF converged?   True
CCSD converged? True
FCI converged?  True

HF energy:   -14.351880476202023
MP2 energy:  -14.376238850841515
CCSD energy: -14.403650751752787
FCI energy:  -14.403655108067682


Number of qubits: 10


In [15]:
taper_hamiltonian = QubitTapering(molecule.H_q)

print(f'We are able to taper {taper_hamiltonian.n_taper} qubits from the Hamiltonian.\n')
print('The symmetry generators are\n')
print(taper_hamiltonian.symmetry_generators)
print('\nand may be rotated via a sequence of Clifford pi/2 rotations\n')
print(taper_hamiltonian.stabilizers.stabilizer_rotations)
print('\nonto the single-qubit Pauli operators\n')
print(taper_hamiltonian.stabilizers.rotate_onto_single_qubit_paulis())
hf_array = molecule.H_fermion.hf_comp_basis_state
taper_hamiltonian.stabilizers.update_sector(hf_array)
print(f'\nThe symetry sector corresponding with the reference {hf_array} is {taper_hamiltonian.stabilizers.coeff_vec}')
ham_tap = taper_hamiltonian.taper_it(ref_state=hf_array)
#sor_tap = taper_hamiltonian.taper_it(aux_operator=sor_ham, ref_state=hf_array)
#sor_tap.coeff_vec/=np.linalg.norm(sor_tap.coeff_vec)
ucc_tap = taper_hamiltonian.taper_it(aux_operator=molecule.T_q, ref_state=hf_array)
n_taper = taper_hamiltonian.n_taper
tapered_qubits   = taper_hamiltonian.stab_qubit_indices
untapered_qubits = taper_hamiltonian.free_qubit_indices
hf_tapered = taper_hamiltonian.tapered_ref_state

print(f'The tapered Hartree-Fock state is', QuantumState([hf_tapered]))

We are able to taper 5 qubits from the Hamiltonian.

The symmetry generators are

1 ZIZIIZIZIZ 
1 IZIZIZIZIZ 
1 IIIIZZIIII 
1 IIIIIIZZII 
1 IIIIIIIIZZ

and may be rotated via a sequence of Clifford pi/2 rotations

[(1.000+0.000j IIIIYZIIII, None), (1.000+0.000j IIIIIIYZII, None), (1.000+0.000j IIIIIIIIYZ, None), (1.000+0.000j YIZIIZIZIZ, None), (1.000+0.000j IYIZIZIZIZ, None)]

onto the single-qubit Pauli operators

-1 XIIIIIIIII 
-1 IXIIIIIIII 
-1 IIIIXIIIII 
-1 IIIIIIXIII 
-1 IIIIIIIIXI

The symetry sector corresponding with the reference [1 1 1 1 0 0 0 0 0 0] is [1 1 1 1 1]
The tapered Hartree-Fock state is  1.0000000000 |11000>


In [16]:
cs_vqe = CS_VQE(ham_tap, hf_tapered, basis_weighting_operator=ucc_tap)

In [17]:
print(cs_vqe.noncontextual_operator)

-8.672+0.000j IIIII +
-0.060+0.000j XIIII +
-0.047+0.000j ZIIII +
-0.047+0.000j IZIII +
0.038-0.000j XZIII +
0.670+0.000j ZZIII +
-0.297+0.000j IIZII +
0.183+0.000j ZIZII +
0.183+0.000j IZZII +
0.010+0.000j XZZII +
-0.297+0.000j IIIZI +
0.183+0.000j ZIIZI +
0.183+0.000j IZIZI +
0.010+0.000j XZIZI +
0.389+0.000j IIZZI +
0.281+0.000j ZIZZI +
0.281+0.000j IZZZI +
-0.010+0.000j XZZZI +
-0.297+0.000j IIIIZ +
0.183+0.000j ZIIIZ +
0.183+0.000j IZIIZ +
0.010+0.000j XZIIZ +
0.389+0.000j IIZIZ +
0.281+0.000j ZIZIZ +
0.281+0.000j IZZIZ +
-0.010+0.000j XZZIZ +
0.389+0.000j IIIZZ +
0.281+0.000j ZIIZZ +
0.281+0.000j IZIZZ +
-0.010+0.000j XZIZZ +
0.263+0.000j IIZZZ +
0.060+0.000j XIZZZ +
2.243-0.000j ZIZZZ +
2.243-0.000j IZZZZ +
-0.038+0.000j XZZZZ +
0.286+0.000j ZZZZZ


In [18]:
H_graph = ObservableGraph(ham_tap.symp_matrix, ham_tap.coeff_vec)

In [43]:
C_cliques = H_graph.clique_cover(clique_relation='C', colouring_strategy='random_sequential')
print(C_cliques[0])

-0.005+0.000j ZYYZZ +
0.005+0.000j YZZYZ +
-0.010+0.000j ZXZII +
0.004-0.000j IZXZZ +
-0.022+0.000j YXYII +
0.005+0.000j IXIIX +
0.005+0.000j IXIXI +
-0.005+0.000j XIXII +
0.012+0.000j IIIXX +
0.022+0.000j YYZZZ +
-8.672+0.000j IIIII +
-0.010+0.000j XZIZZ +
0.005+0.000j YZZZY


In [44]:
C0 = C_cliques[0]
C1 = C_cliques[1]

C_union = C0+C1
C_graph = ObservableGraph(C_union.symp_matrix, C_union.coeff_vec)
print(C0)
print()
print(C1)

AC_cliques = C_graph.clique_cover(clique_relation='AC', colouring_strategy='random_sequential')

-0.005+0.000j ZYYZZ +
0.005+0.000j YZZYZ +
-0.010+0.000j ZXZII +
0.004-0.000j IZXZZ +
-0.022+0.000j YXYII +
0.005+0.000j IXIIX +
0.005+0.000j IXIXI +
-0.005+0.000j XIXII +
0.012+0.000j IIIXX +
0.022+0.000j YYZZZ +
-8.672+0.000j IIIII +
-0.010+0.000j XZIZZ +
0.005+0.000j YZZZY

-0.005+0.000j YIIIY +
-0.022+0.000j XXIIX +
0.010+0.000j XZIIZ +
-0.010+0.000j ZXIIZ +
0.005-0.000j XZXZZ +
0.022-0.000j YYXII +
-0.004+0.000j ZZIIX +
0.005+0.000j IYIIY +
-0.297+0.000j IIIZI +
-0.004+0.000j IIXII +
-0.022+0.000j YYIII +
0.010+0.000j ZXIZZ +
-0.005+0.000j ZXXZZ


In [46]:
print(AC_cliques[0])

0.005+0.000j IYIIY +
-0.010+0.000j ZXZII


In [22]:
import numpy as np
from symred.symplectic import StabilizerOp, find_symmetry_basis

random_Z_block = np.random.randint(0,2,(3, 5))
S = StabilizerOp(np.hstack([np.zeros_like(random_Z_block), random_Z_block]), np.ones(3))
print(S)

1 ZIIZZ 
1 ZZZZZ 
1 IIIZZ


In [23]:
S.update_sector(hf_tapered)

In [24]:
print(S)

-1 ZIIZZ 
1 ZZZZZ 
1 IIIZZ


In [25]:
S.target_sqp = 'X'
print(S.rotate_onto_single_qubit_paulis())

-1 XIIII 
-1 IXIII 
-1 IIIXI


In [26]:
S.target_sqp = 'X'
print(S.rotate_onto_single_qubit_paulis())

-1 XIIII 
-1 IXIII 
-1 IIIXI
